<a href="https://colab.research.google.com/github/copysb/university/blob/main/laba3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ba17996969662527032df4b33eb0845d612343a44b3a29ec6458954d49118031
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

conf = SparkConf().set('spark.ui.port', '4050')\
                  .set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true') #трекер, чтобы возвращать ресурсы
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Создание DataFrame**

In [ ]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true'})\
          .load("/content/drive/MyDrive/user_ratedmovies.dat")

In [ ]:
df

DataFrame[userID: string, movieID: string, rating: string, date_day: string, date_month: string, date_year: string, date_hour: string, date_minute: string, date_second: string]

In [ ]:
df.printSchema()

root
 |-- userID: string (nullable = true)
 |-- movieID: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- date_day: string (nullable = true)
 |-- date_month: string (nullable = true)
 |-- date_year: string (nullable = true)
 |-- date_hour: string (nullable = true)
 |-- date_minute: string (nullable = true)
 |-- date_second: string (nullable = true)



In [ ]:
df.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|      3|     1|      29|        10|     2006|       23|         17|         16|
|    75|     32|   4.5|      29|        10|     2006|       23|         23|         44|
|    75|    110|     4|      29|        10|     2006|       23|         30|          8|
|    75|    160|     2|      29|        10|     2006|       23|         16|         52|
|    75|    163|     4|      29|        10|     2006|       23|         29|         30|
|    75|    165|   4.5|      29|        10|     2006|       23|         25|         15|
|    75|    173|   3.5|      29|        10|     2006|       23|         17|         37|
|    75|    296|     5|      29|        10|     2006|       23|         24|         49|
|    75|    353|   3.5|      29|

Все в string, так не пойдет, давайте автоматически определим тип данных

In [ ]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true', 'inferSchema': 'true'})\
          .load("user_ratedmovies.dat")

In [ ]:
df

DataFrame[userID: int, movieID: int, rating: double, date_day: int, date_month: int, date_year: int, date_hour: int, date_minute: int, date_second: int]

In [ ]:
df.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|      3|   1.0|      29|        10|     2006|       23|         17|         16|
|    75|     32|   4.5|      29|        10|     2006|       23|         23|         44|
|    75|    110|   4.0|      29|        10|     2006|       23|         30|          8|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|    163|   4.0|      29|        10|     2006|       23|         29|         30|
|    75|    165|   4.5|      29|        10|     2006|       23|         25|         15|
|    75|    173|   3.5|      29|        10|     2006|       23|         17|         37|
|    75|    296|   5.0|      29|        10|     2006|       23|         24|         49|
|    75|    353|   3.5|      29|

А можно заранее сказать, какой тип данных я ожидаю?

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [ ]:
schema = StructType([ \
    StructField("userID",StringType(),True), \
    StructField("movieID",IntegerType(),True), \
    StructField("rating",DoubleType(),True), \
    StructField("date_day", StringType(), True), \
    StructField("date_month", StringType(), True), \
    StructField("date_year", IntegerType(), True), \
    StructField("date_hour", IntegerType(), True), \
    StructField("date_minute", IntegerType(), True), \
    StructField("date_second", IntegerType(), True)
  ])

In [ ]:
schema

StructType([StructField('userID', StringType(), True), StructField('movieID', IntegerType(), True), StructField('rating', DoubleType(), True), StructField('date_day', StringType(), True), StructField('date_month', StringType(), True), StructField('date_year', IntegerType(), True), StructField('date_hour', IntegerType(), True), StructField('date_minute', IntegerType(), True), StructField('date_second', IntegerType(), True)])

In [ ]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true'})\
          .schema(schema)\
          .load("/content/drive/MyDrive/Преподавание/ОЦГ/user_ratedmovies.dat")

In [ ]:
df.printSchema()

root
 |-- userID: string (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: string (nullable = true)
 |-- date_month: string (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



In [ ]:
df.show()

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|      3|   1.0|      29|        10|     2006|       23|         17|         16|
|    75|     32|   4.5|      29|        10|     2006|       23|         23|         44|
|    75|    110|   4.0|      29|        10|     2006|       23|         30|          8|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|    163|   4.0|      29|        10|     2006|       23|         29|         30|
|    75|    165|   4.5|      29|        10|     2006|       23|         25|         15|
|    75|    173|   3.5|      29|        10|     2006|       23|         17|         37|
|    75|    296|   5.0|      29|        10|     2006|       23|         24|         49|
|    75|    353|   3.5|      29|

Но есть уже готовая обертка под все нужды

In [ ]:
df = spark.read.csv(path='/content/drive/MyDrive/Преподавание/ОЦГ/user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [ ]:
df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: integer (nullable = true)
 |-- date_month: integer (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



Так, а как сохранить? Лучше быть аккуратнее с overwrite, перезапишет весь указанный путь, append будет безопаснее

In [ ]:
df.write.option("header",True)\
        .mode("overwrite")\
        .parquet('write_1.parquet')

А что с партицированием?

In [ ]:
df.write.option("header",True)\
        .partitionBy('date_year')\
        .mode("overwrite")\
        .parquet('write_2.parquet')

In [ ]:
spark.read.parquet('/content/write_2.parquet/date_year=2002').show()

+------+-------+------+--------+----------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+-----------+-----------+
|  1224|      1|   5.0|      31|         5|        4|         54|          9|
|  1224|      3|   4.0|      31|         5|        5|          9|         19|
|  1224|      6|   5.0|      31|         5|        4|         38|         50|
|  1224|     10|   4.0|      31|         5|        4|         43|         13|
|  1224|     36|   3.0|       5|         6|        7|         19|          4|
|  1224|     50|   5.0|      31|         5|        3|         40|          9|
|  1224|     86|   3.0|      31|         5|        5|         36|         37|
|  1224|    112|   4.0|      31|         5|        4|         44|         11|
|  1224|    153|   2.0|       1|         6|        4|         31|         57|
|  1224|    161|   4.0|      31|         5|        4|          2

Кстати, раз уж заговорили про схемы данных, то их можно задавать интереснее, например, под группированные данные

In [ ]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])
df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------------+-----+------+------+
|name                |id   |gender|salary|
+--------------------+-----+------+------+
|{James, , Smith}    |36636|M     |3100  |
|{Michael, Rose, }   |40288|M     |4300  |
|{Robert, , Williams}|42114|M     |1400  |
|{Maria, Anne, Jones}|39192|F     |5500  |
|{Jen, Mary, Brown}  |     |F     |-1    |
+--------------------+-----+------+------+



Со структурой можно работать и менять ее под ваши нужны

In [ ]:
updatedDF = df2.withColumn("OtherInfo",
    F.struct(F.col("id").alias("identifier"),
    F.col("gender").alias("gender"),
    F.col("salary").alias("salary"),
    F.when(F.col("salary").cast(IntegerType()) < 2000,"Low")
      .when(F.col("salary").cast(IntegerType()) < 4000,"Medium")
      .otherwise("High").alias("Salary_Grade")
  )).drop("id","gender","salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- OtherInfo: struct (nullable = false)
 |    |-- identifier: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- salary: integer (nullable = true)
 |    |-- Salary_Grade: string (nullable = false)

+--------------------+------------------------+
|name                |OtherInfo               |
+--------------------+------------------------+
|{James, , Smith}    |{36636, M, 3100, Medium}|
|{Michael, Rose, }   |{40288, M, 4300, High}  |
|{Robert, , Williams}|{42114, M, 1400, Low}   |
|{Maria, Anne, Jones}|{39192, F, 5500, High}  |
|{Jen, Mary, Brown}  |{, F, -1, Low}          |
+--------------------+------------------------+



Что мы там сделали????

1) Создали новую структуру данных OtherInfo

2) Передали туда id (переименовав столбец), gender, salary

3) Создали столбец Salary_grade из условий

4) удалили id, gender, salary из старой структуры

Есть и еще структуры данных!

In [ ]:
from pyspark.sql.types import ArrayType, MapType

In [ ]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

In [ ]:
structureData = [
    (("James","","Smith"), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (("Michael","Rose",""), ['car', 'football'], {2: 'b'}),
    (("Robert","","Williams"), ['box', 'music'], {3: 'c'})
  ]

In [ ]:
df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

In [ ]:
df3.select('properties').collect()

**Описание данных**

Общее описание данных

In [ ]:
df = spark.read.csv(path='/content/drive/MyDrive/Преподавание/ОЦГ/user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [ ]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|summary|           userID|           movieID|            rating|          date_day|        date_month|        date_year|         date_hour|       date_minute|       date_second|
+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|  count|           855598|            855598|            855598|            855598|            855598|           855598|            855598|            855598|            855598|
|   mean|35190.83255103448| 8710.179402008887| 3.437945156487042|15.568923723524366|  6.54079485926802|2005.323717446745|12.124531614145896|29.645382527775894|29.510222090280717|
| stddev|20385.00334699137|14446.852908494242|1.0025608721610357| 8.951201361275226|3.5063994259089704|2.

In [ ]:
df.summary().show()

+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|summary|           userID|           movieID|            rating|          date_day|        date_month|        date_year|         date_hour|       date_minute|       date_second|
+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|  count|           855598|            855598|            855598|            855598|            855598|           855598|            855598|            855598|            855598|
|   mean|35190.83255103448| 8710.179402008887| 3.437945156487042|15.568923723524366|  6.54079485926802|2005.323717446745|12.124531614145896|29.645382527775894|29.510222090280717|
| stddev|20385.00334699137|14446.852908494242|1.0025608721610357| 8.951201361275226|3.5063994259089704|2.

Количество записей

In [ ]:
df.count()

Количество партиций

In [ ]:
df.rdd.getNumPartitions()

 Менять число партиций можно, все как с rdd

In [ ]:
df = df.repartition(5)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df = df.coalesce(2)

In [ ]:
df.rdd.getNumPartitions()

**Различные методы**

Ну теперь давайте тыкать

Удаляем дубликаты и помним, что есть actions и transformations, count заставит все сделать

In [ ]:
df.unique()

In [ ]:
df_without_duplicates = df.drop_duplicates()

Есть alias

In [ ]:
df_without_duplicates = df.dropDuplicates()

Как удалить дубликаты по отдельным колонкам?

In [ ]:
df_without_duplicates = df.drop_duplicates(['userID', 'rating'])

In [ ]:
df_without_duplicates.count()

18847

In [ ]:
df_without_duplicates.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  4944|      2|   1.5|      22|        10|     2007|       20|         28|         23|
| 11930|      2|   3.0|       2|         4|     2005|       16|         41|          5|
| 12356|    420|   2.5|      26|         1|     2006|        3|         29|          9|
| 13014|     44|   2.0|      21|         6|     2003|        7|         46|         42|
| 15715|      1|   5.0|       7|         2|     2006|        1|         26|          6|
| 15816|      6|   4.5|       4|         1|     2007|       22|         13|         58|
| 16862|      6|   4.5|      10|         2|     2006|        5|         14|          0|
| 19519|      1|   3.0|      18|         4|     2006|       23|         48|          9|
| 19838|      2|   3.5|      20|

Как закинуть данные в любимый pandas?

 Самый простой вариант - встроенный метод, но он жутко медленный при существенном объеме данных. Лучше сохранить паркет и считать через pd.read_parquet()

In [ ]:
import pandas as pd

In [ ]:
%%timeit

pandas_df = df.toPandas()

In [ ]:
pandas_df = df.toPandas()
pandas_df

In [ ]:
pandas_df.info()

In [ ]:
%%timeit

pandas_df = pd.read_parquet('write_1.parquet')

40 ms ± 4.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
pandas_df = pd.read_parquet('write_1.parquet')
pandas_df

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30
...,...,...,...,...,...,...,...,...,...
855593,71534,44555,4.0,3,12,2007,3,5,38
855594,71534,46578,4.0,3,12,2007,2,56,44
855595,71534,48516,4.5,3,12,2007,2,53,46
855596,71534,61075,5.0,10,10,2008,9,56,5


In [ ]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855598 entries, 0 to 855597
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userID       855598 non-null  int32  
 1   movieID      855598 non-null  int32  
 2   rating       855598 non-null  float64
 3   date_day     855598 non-null  int32  
 4   date_month   855598 non-null  int32  
 5   date_year    855598 non-null  int32  
 6   date_hour    855598 non-null  int32  
 7   date_minute  855598 non-null  int32  
 8   date_second  855598 non-null  int32  
dtypes: float64(1), int32(8)
memory usage: 32.6 MB


**Show**

In [ ]:
df.show(10)

Обрезаем до 2 символов

In [ ]:
df.show(10, truncate=2)

вертикальное отображение

In [ ]:
df.show(10, vertical=True)

**Select**

В PySpark функция select() используется для выбора одного, нескольких столбцов по индексу, всех столбцов из списка и вложенных столбцов из фрейма данных. Функция PySpark select() является функцией преобразования, поэтому она возвращает новый фрейм данных с выбранными столбцами.

In [ ]:
df.columns

['userID',
 'movieID',
 'rating',
 'date_day',
 'date_month',
 'date_year',
 'date_hour',
 'date_minute',
 'date_second']

Упс, pandas-style тут не приветствуется

In [ ]:
df.loc[:, ['userID', ;sdflsdfj]]
df[['userID', '']]

In [ ]:
df.userID.show(5)

In [ ]:
df.select('userID').show(5)

Куча вариантов, выбирайте любой

In [ ]:
df.select('userID', 'rating').show(5)

In [ ]:
df.select(['userID', 'rating']).show(5)

In [ ]:
df.select(df.userID,df.rating).show(5)

In [ ]:
df.select(df['userID'],df['rating']).show(5)

In [ ]:
df.select(F.col("userID"), F.col("rating")).show(5)

можно налету переименовать столбец

In [ ]:
df.select(df.userID, df.rating.alias('mark')).show(5)

In [ ]:
#регулярки
df.select(df.colRegex("`d+.*y`")).show(5)

примеры с другим датафреймом, где структура сложнее

In [ ]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

In [ ]:
df2.select('name').show(5)

In [ ]:
df2.select('name.lastname').show(5)

In [ ]:
df2.select('name.firstname', 'name.lastname').show(5)

**withColumn**

PySpark withColumn() - это функция преобразования (transform), которая используется для изменения значения, преобразования типа данных существующего столбца, создания нового столбца и многого другого. Поговорим о часто используемых операциях со столбцами данных PySpark, используя примеры.

In [ ]:
df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: integer (nullable = true)
 |-- date_month: integer (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



Меняем тип данных

In [ ]:
df['new_col'] = []

In [ ]:
df.withColumn("date_month", F.col("date_month").cast("String")).printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: integer (nullable = true)
 |-- date_month: string (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



Модифицировать столбец/создать новый

In [ ]:
df.withColumn("rating_x_10",F.col("rating") * 10).show(5)

Делаем 2 константных столбца

In [ ]:
df.withColumn('fix_1', F.lit(1)).withColumn('fix_2', F.lit(2)).show(5)

**withColumnsRenamed**

Предыдущий вариант не давал возможности переименовать столбцы, это можно сделать иначе

In [ ]:
df.withColumnRenamed('rating', 'mark').show(5)

**filter (where) и иные филтрации**

Функция PySpark filter() используется для фильтрации строк из RDD / DataFrame на основе заданного условия или выражения SQL, вы также можете использовать предложение where() вместо filter() обе эти функции работают аналогично.

1 условие

In [ ]:
df.filter(df.rating == 5.0)

DataFrame[userID: int, movieID: int, rating: double, date_day: int, date_month: int, date_year: int, date_hour: int, date_minute: int, date_second: int]

In [ ]:
df.filter((df.rating == 5.0) & (df.rating == 1))

In [ ]:
df.filter(~(df.rating == 5.0)).show(5)

In [ ]:
df.filter('rating = 5').show(5)

DataFrame[userID: int, movieID: int, rating: double, date_day: int, date_month: int, date_year: int, date_hour: int, date_minute: int, date_second: int]

Несколько условий

In [ ]:
df.filter((df.rating == 5.0) & (df.date_year == 2006)).show(5)

In [ ]:
df.filter('(rating = 5.0) and (date_year = 2006)').show(5)

In [ ]:
df.filter('(rating = 5.0) and (userID between 70 and 80)').show(5)

фильтр по списку значений из list

In [ ]:
years = [2006, 2007]

In [ ]:
df.filter(df.date_year.isin(years)).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|      3|   1.0|      29|        10|     2006|       23|         17|         16|
|    75|     32|   4.5|      29|        10|     2006|       23|         23|         44|
|    75|    110|   4.0|      29|        10|     2006|       23|         30|          8|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|    163|   4.0|      29|        10|     2006|       23|         29|         30|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



 проверим

In [ ]:
df.collect()

In [ ]:
df.filter(df.date_year.isin(years))\
  .select('date_year')\
  .dropDuplicates()\
  .collect()

[Row(date_year=2007), Row(date_year=2006)]

создадим игрушечный датайфрейм для текстовых столбцов

In [ ]:
data2 = [(2,"Michael Rose"),(3,"Robert Williams"),
     (4,"Rames Rose"),(5,"Rames Black"), (6, 'Albus Torch'),
     (7, 'Fred Tf')
  ]
df2 = spark.createDataFrame(data2, ['id', 'name'])

In [ ]:
df2.show()

In [ ]:
df2.filter('name like "R%"').show()

In [ ]:
df2.filter(df2.name.startswith('R')).show()

In [ ]:
df2.filter(df2.name.endswith('Tf')).show()

In [ ]:
df2.filter(df2.name.contains('Wil')).show()

Бывает, что у нас внутри датафрейма есть массив и с ним что-то хочется сделать

In [ ]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

structureData = [
    (("James","","Smith"), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (("Michael","Rose",""), ['car', 'football'], {2: 'b'}),
    (("Robert","","Williams"), ['box', 'music'], {3: 'c'})
  ]

df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

In [ ]:
df3.filter(F.array_contains(df3.hobbies,"football")) \
    .show()

**Сортировка**

сделаем еще фильтрацию, чтобы увидеть результат (orderBy тут аналог)

In [ ]:
df.filter(df.userID == 75).sort(df.date_minute, df.rating.desc()).show(20)

**groupby**

Когда мы выполняем groupBy() в PySpark DataFrame, он возвращает объект GroupedData, который содержит следующие агрегатные функции:

min(), max(), mean(), count(), sum(), avg(), agg(), pivot()

In [ ]:
df.groupby(['date_year', 'date_month']).mean('rating').collect()

мы уже умеем применять разные методы

In [ ]:
df.groupby('date_year')\
  .mean('rating')\
  .sort('date_year')\
  .collect()

In [ ]:
df.filter(df.rating <= 2)\
  .groupby('date_year')\
  .count()\
  .withColumnRenamed('count', 'number')\
  .sort('date_year')\
  .collect()

несколько колонок

In [ ]:
df.groupBy("date_year", "date_month") \
  .mean("rating", "userID") \
  .sort('date_year', 'date_month')\
  .show()

Для того, чтобы делать несколько разных агрегаций и еще менять сразу имя столбца нужно немного изменить синтаксис

И стоит помнить, что персентили тут считаются приближенно

In [ ]:
df.groupBy("date_year") \
    .agg(F.min("rating").alias("min_rating"), \
         F.mean("rating").alias("mean_rating"), \
         F.max("rating").alias("max_rating"),
         F.percentile_approx("rating", 0.5).alias("median")
         ) \
    .show()

+---------+----------+------------------+----------+------+
|date_year|min_rating|       mean_rating|max_rating|median|
+---------+----------+------------------+----------+------+
|     2003|       0.5|3.4279154852282083|       5.0|   3.5|
|     2007|       0.5|3.4367384383270894|       5.0|   3.5|
|     2006|       0.5|3.4294610550371356|       5.0|   3.5|
|     1997|       1.0| 3.571018651362984|       5.0|   4.0|
|     2004|       0.5|3.3916603359500295|       5.0|   3.5|
|     1998|       1.0|3.5080568720379146|       5.0|   4.0|
|     2009|       0.5| 3.369243156199678|       5.0|   3.5|
|     2001|       1.0|3.4490124195720484|       5.0|   4.0|
|     2005|       0.5| 3.395790002433071|       5.0|   3.5|
|     2000|       1.0|3.5767733835530446|       5.0|   4.0|
|     2008|       0.5| 3.486752231109465|       5.0|   3.5|
|     1999|       1.0|3.4925797284496367|       5.0|   4.0|
|     2002|       1.0|3.4140127388535033|       5.0|   4.0|
+---------+----------+------------------

Еще можно сделать pivot

In [ ]:
df.groupBy('date_year')\
  .pivot('date_month')\
  .mean('rating')\
  .show(5)

**Join's**

Куда же без них. Что есть: INNER, LEFT OUTER, RIGHT OUTER, LEFT ANTI, LEFT SEMI, CROSS, SELF JOIN

Благодаря оптимизации в датафреймах уже все хорошо работает, спасибо catalist, но чудеса не вечны и плохой код/незнание данных все равно даст о себе знать

Сделаем для себя несколько таблиц, чтобы можно было экспериментировать

In [ ]:
df_mean_user_rating = df.groupBy("userID")\
                        .mean('rating')\
                        .withColumnRenamed('avg(rating)', 'avg_rating_all')

df_mean_user_rating_year = df.groupby('userID', 'date_year')\
                             .mean('rating')\
                             .withColumnRenamed('avg(rating)', 'avg_rating_year')

In [ ]:
df_mean_user_rating.printSchema()

df_mean_user_rating_year.printSchema()

И давайте все в 1 блоке кода, чтобы не растягивать

In [ ]:
df.join(df_mean_user_rating, on=df.userID == df_mean_user_rating.userID, how='inner')\
  .join(df_mean_user_rating_year, on=[df.userID == df_mean_user_rating_year.userID,
                                      df.date_year == df_mean_user_rating_year.date_year],
        how='inner')

Надо удалить дублирующиеся столбцы

In [ ]:
res_join = df.alias('t').join(df_mean_user_rating.alias('t1'), on=F.col('t.userID') == F.col('t1.userID'), how='inner')\
  .drop(F.col('t1.userID'))\
  .join(df_mean_user_rating_year.alias('t2'), on=[F.col('t.userID') == F.col('t2.userID'),
                                      F.col('t.date_year') == F.col('t2.date_year')],
        how='inner')\
  .drop(F.col('t2.userID'))\
  .drop(F.col('t2.date_year'))

In [ ]:
res_join.show(5)

**union и unionAll**

Используются для объединения датафреймов с одинаковой структурой, используется union, так как unionAll с версии 2.0.0 более не используется

In [ ]:
df1 = df.filter(df.date_year == 2006)
df2 = df.filter(df.date_year != 2006)

In [ ]:
union_df = df1.union(df2)

print(df.count(), union_df.count())

Desclaimer: все по sql, надеюсь, помнят разницу между union и union all, когда union убирает дубликаты. Так вот pyspark ничего не удаляет, убрать дубликаты можно только через drop_duplicates, distinct

Также при union pyspark делает объединение по столбцам as is, не пытаясь понять, что в одном датафрейме нужный стобец на 1 позиции, а в другом он на 5. Для этого с версии 3.1 есть замечательный метод unionByName

**SQL**

Ну раз уж пошла такая тема, давайте рассмотрим, как можно сделать все при помощи любимого SQL

можно делать TempView и GlodalTempView, отличие в том, что обычный view будет жить, пока жива сессия спрака, а глобальная, пока жив sparkcontext

In [ ]:
df.createOrReplaceTempView('df')

In [ ]:
query = '''
select
    userID, movieID, rating, udf_example_decorator(rating) as rating_100
from
    df
'''
spark.sql(query).show(5)

Ну и наш join

In [ ]:
df_mean_user_rating.createOrReplaceTempView('df_mean_user_rating')
df_mean_user_rating_year.createOrReplaceTempView('df_mean_user_rating_year')

In [ ]:
query = '''
select t.*, t1.avg_rating_all, t2.avg_rating_year
from
df t, df_mean_user_rating t1, df_mean_user_rating_year t2
where
    t.userID = t1.userID and
    t.userID = t2.userID and
    t.date_year = t2.date_year
'''
spark.sql(query).show(5)

**fill() и fillna()**

Оба метода идентичны, заполняют пропуски

In [ ]:
import numpy as np

In [ ]:
data2 = [(2,"Michael Rose"),(3,"Robert Williams"),
     (4,"Rames Rose"),(5, None), (6, None),
     (None, 'Fred Tf')
  ]
df2 = spark.createDataFrame(data2, ['id', 'name'])

А где пропуски?

In [ ]:
df2.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+---+----+
| id|name|
+---+----+
|  1|   2|
+---+----+



In [ ]:
df2.show()

In [ ]:
df2.fillna({'id': 0}).show()

+---+---------------+
| id|           name|
+---+---------------+
|  2|   Michael Rose|
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|           NULL|
|  6|           NULL|
|  0|        Fred Tf|
+---+---------------+



In [ ]:
df2.fillna({'id': 0, 'name': 'Unknown'}).show()

Аналогично

In [ ]:
df2.na.fill({'id': 0, 'name': 'Unknown'}).show()

Немного поговорим о том, как делаеть подвыборки

**sample и sampleBy**

Не забываем про возможную некоторую недетерминированность

In [ ]:
import pyspark.sql.functions as F

In [ ]:
trans_data = spark.read.parquet('/content/drive/MyDrive/Преподавание/ОЦГ/spark_transactions.parquet')
trans_data = trans_data.withColumn('target', F.when(F.col('IsFraud')=='No', 0).otherwise(1))

In [ ]:
trans_data.count()

24386900

In [ ]:
trans_data.select(F.mean(F.col('target'))).show()

+--------------------+
|         avg(target)|
+--------------------+
|0.001220204290008...|
+--------------------+



In [ ]:
trans_data.select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+--------+
|target|   count|
+------+--------+
|     1|   29757|
|     0|24357143|
+------+--------+



In [ ]:
trans_data_simple = trans_data.sample(withReplacement=False, fraction=0.1, seed=3)
print(trans_data_simple.count())
trans_data_simple.select(F.mean(F.col('target'))).show()

2439262
+-------------------+
|        avg(target)|
+-------------------+
|0.00120897222192614|
+-------------------+



In [ ]:
from pyspark.sql.window import Window

In [ ]:
trans_data_simple.select('User', 'Card', 'Year', 'Month', 'Day')\
.withColumn('rn_1', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc())))\
.withColumn('rn_2', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc(), F.col('Month').desc())))\
.withColumn('mean', F.mean('raiting').over(Window.partitionBy('date_year')))\
.withColumn('lag', F.lag('Day').over(Window.partitionBy('User', 'Card', 'Month').orderBy(F.col('Year').asc(),
                                                                                         F.col('Month').desc(),
                                                                                         F.col('Day').desc())))\
.show()

+----+----+----+-----+---+----+----+------------------+----+
|User|Card|Year|Month|Day|rn_1|rn_2|              mean| lag|
+----+----+----+-----+---+----+----+------------------+----+
|   0|   1|2015|    1| 24|  19|  37|              13.6|NULL|
|   0|   1|2015|    1| 16|  18|  36|              13.6|  24|
|   0|   1|2016|    1|  8|  41|  57|              13.6|  16|
|   0|   1|2016|    1|  5|  40|  56|              13.6|   8|
|   0|   1|2016|    1|  3|  39|  55|              13.6|   5|
|   0|   1|2016|    1|  1|  38|  54|              13.6|   3|
|   0|   1|2018|    1| 27|  74|  88|              13.6|   1|
|   0|   1|2018|    1|  5|  73|  87|              13.6|  27|
|   0|   1|2020|    1| 26| 102| 105|              13.6|   5|
|   0|   1|2020|    1| 21| 101| 104|              13.6|  26|
|   0|   1|2015|    2| 15|  20|  35|15.666666666666666|NULL|
|   0|   1|2016|    2|  7|  42|  53|15.666666666666666|  15|
|   0|   1|2018|    2| 23|  76|  86|15.666666666666666|   7|
|   0|   1|2018|    2|  

In [ ]:
df.groupby('date_year').mean('rating').show()

+---------+------------------+
|date_year|       avg(rating)|
+---------+------------------+
|     2003|3.4279154852282083|
|     2007|3.4367384383270894|
|     2006|3.4294610550371356|
|     1997| 3.571018651362984|
|     2004|3.3916603359500295|
|     1998|3.5080568720379146|
|     2009| 3.369243156199678|
|     2001|3.4490124195720484|
|     2005| 3.395790002433071|
|     2000|3.5767733835530446|
|     2008| 3.486752231109465|
|     1999|3.4925797284496367|
|     2002|3.4140127388535033|
+---------+------------------+



In [ ]:
df\
.withColumn('mean', F.mean('rating').over(Window.partitionBy('date_year'))).show(100)


Сборка последовательностей через **collect_list()**

In [ ]:
trans_data.printSchema()

In [ ]:
trans_data_simple_seq = trans_data_simple.select('User', 'Card', F.struct('Amount', 'Day').alias('sequence'))

In [ ]:
trans_data_simple_seq.show()

In [ ]:
trans_data_simple_seq = trans_data_simple_seq.groupBy('User', 'Card')\
                                             .agg(F.collect_list('sequence').alias('sequence'))

In [ ]:
trans_data_simple_seq.show(truncate=50)

А что с этим делать дальше? Можно перейти к RDD и построчно обрабатывать

In [ ]:
row = trans_data_simple_seq.rdd.take(1)

In [ ]:
row

In [ ]:
row[0]['sequence']

**Домашнее задание**

Куда же без домашки, верно?

Есть данные по транзакциям клиентов, ваша задача состоит в анализе этих данных и подготовки к структуре, которая похожа на ту структуру, которая сейчас часто нами используется при построении моделей на транзакциях + промежуточные задания.

Не забудьте делать всякие show после каждого задания, чтобы было видно результат

**Файл spark_transactions.parquet можете забрать в папке**

**Важно**
В домашнем задании старайтесь использовать максимально dataframe api, а не sql запросы.

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F
trans_data = spark.read.parquet('/content/drive/MyDrive/spark_transactions.parquet')

In [ ]:
trans_data.count()

24386900

In [ ]:
trans_data.show(5)

+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|User|Card|Year|Month|Day| Time| Amount|          UseChip| MerchantCity|MerchantState|    Zip| MCC|IsFraud|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|   0|   0|2002|    9|  1|06:21|$134.09|Swipe Transaction|     La Verne|           CA|91750.0|5300|     No|
|   0|   0|2002|    9|  1|06:42| $38.48|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|06:22|$120.34|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|17:45|$128.95|Swipe Transaction|Monterey Park|           CA|91754.0|5651|     No|
|   0|   0|2002|    9|  3|06:23|$104.71|Swipe Transaction|     La Verne|           CA|91750.0|5912|     No|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
only showing top 5 rows



Посмотрим на схему данных

In [ ]:
trans_data.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: double (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)



Сколько в среднем транзакций у пользователя

In [ ]:
trans_data.groupBy('User').count().agg({'count': 'avg'}).show()

+----------+
|avg(count)|
+----------+
|  12193.45|
+----------+



Сколько карт у пользователей в среднем

In [ ]:
from pyspark.sql.functions import countDistinct

trans_data.groupBy('User').agg(countDistinct('card').alias('cardcount')).agg({'cardcount': 'avg'}).show()

+--------------+
|avg(cardcount)|
+--------------+
|        3.0695|
+--------------+



Немного обработаем данные: Amount в float, из Time вытянем час транзакции и удалим исходный Time, Zip  к типу int

In [ ]:
from pyspark.sql.functions import col, split, regexp_replace

trans_data_new = trans_data.withColumn('Zip', col('Zip').cast('Float'))
trans_data_new = trans_data_new.withColumn('Amount', regexp_replace(col('Amount'), '[^\d\.]', '').cast('Float'))
trans_data_new = trans_data_new.withColumn('Hour_trans', split(col('Time'), ':')[0])
trans_data_new = trans_data_new.drop('Time')
trans_data_new.printSchema()
trans_data = trans_data_new


root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Amount: float (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: float (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)
 |-- Hour_trans: string (nullable = true)



Посчитайте количество транзакций по годам, учитывая только те транзакции, объем которых был больше 100

In [ ]:
from pyspark.sql.functions import count

filter_data = trans_data_new.filter(trans_data_new.Amount > 100)
year_td = filter_data.groupBy('Year').agg(count('*').alias('trans_count'))
year_td.orderBy('Year').show()


+----+-----------+
|Year|trans_count|
+----+-----------+
|1991|        279|
|1992|        702|
|1993|       1112|
|1994|       1635|
|1995|       2270|
|1996|       3362|
|1997|       5927|
|1998|       9527|
|1999|      14530|
|2000|      22124|
|2001|      32272|
|2002|      43200|
|2003|      57387|
|2004|      73800|
|2005|      92831|
|2006|     111776|
|2007|     128246|
|2008|     145146|
|2009|     158605|
|2010|     174482|
+----+-----------+
only showing top 20 rows



Определите, есть ли пропуски в данных по каждому столбцу

In [ ]:
from pyspark.sql.functions import isnan, when

trans_data.select([count(when(col(c).isNull(), c)).alias(c) for c in trans_data.columns]).show()

+----+----+----+-----+---+------+-------+------------+-------------+-------+---+-------+----------+
|User|Card|Year|Month|Day|Amount|UseChip|MerchantCity|MerchantState|    Zip|MCC|IsFraud|Hour_trans|
+----+----+----+-----+---+------+-------+------------+-------------+-------+---+-------+----------+
|   0|   0|   0|    0|  0|     0|      0|           0|      2720821|2878135|  0|      0|         0|
+----+----+----+-----+---+------+-------+------------+-------------+-------+---+-------+----------+



Заполните пропуски исходя из типа данных

In [ ]:
from pyspark.sql.functions import avg

trans_data = trans_data.fillna('Unknown', subset=['MerchantState'])
val_mean = trans_data.select(avg("Zip")).collect()[0][0]
trans_data = trans_data.fillna(val_mean)

trans_data.show(20)

+----+----+----+-----+---+------+------------------+-------------+-------------+--------+----+-------+----------+
|User|Card|Year|Month|Day|Amount|           UseChip| MerchantCity|MerchantState|     Zip| MCC|IsFraud|Hour_trans|
+----+----+----+-----+---+------+------------------+-------------+-------------+--------+----+-------+----------+
|   0|   0|2002|    9|  1|134.09| Swipe Transaction|     La Verne|           CA| 91750.0|5300|     No|        06|
|   0|   0|2002|    9|  1| 38.48| Swipe Transaction|Monterey Park|           CA| 91754.0|5411|     No|        06|
|   0|   0|2002|    9|  2|120.34| Swipe Transaction|Monterey Park|           CA| 91754.0|5411|     No|        06|
|   0|   0|2002|    9|  2|128.95| Swipe Transaction|Monterey Park|           CA| 91754.0|5651|     No|        17|
|   0|   0|2002|    9|  3|104.71| Swipe Transaction|     La Verne|           CA| 91750.0|5912|     No|        06|
|   0|   0|2002|    9|  3| 86.19| Swipe Transaction|Monterey Park|           CA| 91755.0

При помощи оконных функций для каждого клиента рассчитайте средний размер транзакции, количество транзакций и последнюю по дате транзакцию.

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import avg, count, col, max, concat, lit

df = trans_data.withColumn('date', concat(col('Year'), lit('-'), col('Month'), lit('-'), col('Day')))
window = Window.partitionBy('User')
trans_amount = avg('Amount').over(window).alias('trans_amount')
trans_count = count('Amount').over(window).alias('trans_count')
last_time_trans = max('date').over(window).alias('last_time_trans')

result_df = df.withColumn('trans_amount', trans_amount) \
                      .withColumn('trans_count', trans_count) \
                      .withColumn('last_time_trans', last_time_trans) \
                      .select('User', 'trans_amount', 'trans_count', 'last_time_trans')


result_df = result_df.dropDuplicates(['User'])

result_df.show(5)

+----+------------------+-----------+---------------+
|User|      trans_amount|trans_count|last_time_trans|
+----+------------------+-----------+---------------+
|   0| 90.17621047155345|      19963|       2020-2-9|
|   6| 40.97650493320776|      28237|       2020-2-9|
|   7| 56.84398290211872|        236|       2020-2-9|
|  19| 50.46217367973611|      49584|       2020-2-9|
|  22|40.940002417526635|      16458|       2020-2-9|
+----+------------------+-----------+---------------+
only showing top 5 rows



Теперь самое время сгруппировать данные по каждому клиенту (можно использовать collect_list для сбора данных после агрегации)
Когда будете делать агрегацию, то возьмите только часть выборки, например, через sample, для всей выборки либо не хватит памяти, либо очень долго считать

In [ ]:
from pyspark.sql.functions import collect_list

sampled_data = trans_data.limit(100000)
sampled_data = sampled_data.orderBy("Year", "Month", "Day", "Hour_trans")
grouped_data = sampled_data.groupBy('User', 'Card').agg(collect_list("Amount").alias("Amount_list"),
                                                collect_list("Year").alias("Year_list"),
                                                collect_list("Month").alias("Month_list"),
                                                collect_list("Day").alias("Day_list"),
                                                collect_list("Hour_trans").alias("Hour_list"),
                                                collect_list("UseChip").alias("UseChip_list"),
                                                collect_list("MerchantCity").alias("MerchantCity_list"),
                                                collect_list("Zip").alias("Zip_list"),
                                                collect_list("MCC").alias("MCC_list"),
                                                collect_list("IsFraud").alias("IsFraud"))

grouped_data = grouped_data.orderBy("User", "Card")

grouped_data.show(10)

+----+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|User|Card|         Amount_list|           Year_list|          Month_list|            Day_list|           Hour_list|        UseChip_list|   MerchantCity_list|            Zip_list|            MCC_list|             IsFraud|
+----+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   0|   0|[134.09, 38.48, 1...|[2002, 2002, 2002...|[9, 9, 9, 9, 9, 9...|[1, 1, 2, 2, 3, 3...|[06, 06, 06, 17, ...|[Swipe Transactio...|[La Verne, Monter...|[91750.0, 91754.0...|[5300, 5411, 5411...|[No, No, No, No, ...|
|   0|   1|[56.29, 41.67, 42...|[2014, 2014, 2014...|[4, 4, 4, 4, 4, 4...|[9, 10, 11, 14, 1...|[13, 06, 09, 06, 

Напишите python функцию, которая возьмет данные после агрегации последовательностей, отсортирует их внутри по дате и времени и преобразует к формату python dict:
{'User': User,
'Card': Card,
'sequence':{
    'amount': [последовательность],
    'year': [последовательность],
    'month': [последовательность],
    'day': [последовательность],
    'time': [последовательность],
    'MCC': [последовательность]
}
}




Выведите как пример одну преобразованную запись, результаты сохраните на диск в  parquet

In [ ]:
from pyspark.sql.functions import sort_array, array, col

def process_and_save_data(df, output_path):

    def row_to_dict(row):
        sequence_dict = {
            "amount": row.Amount_list,
            "year": row.Year_list,
            "month": row.Month_list,
            "day": row.Day_list,
            "hour": row.Hour_list,
            "MCC": row.MCC_list
        }
        return {
            "User": row.User,
            "Card": row.Card,
            "sequence": sequence_dict
        }

    sampled_result = df.rdd.map(row_to_dict).collect()

    print("Пример:")
    print(sampled_result[0])

    spark.createDataFrame(sampled_result).write.parquet(output_path, mode='overwrite')

process_and_save_data(grouped_data, "processed_data.parquet")


Пример:
{'User': 0, 'Card': 0, 'sequence': {'amount': [134.08999633789062, 38.47999954223633, 120.33999633789062, 128.9499969482422, 104.70999908447266, 86.19000244140625, 93.83999633789062, 123.5, 61.720001220703125, 57.099998474121094, 76.06999969482422, 53.90999984741211, 110.37000274658203, 117.05000305175781, 45.29999923706055, 29.34000015258789, 147.4499969482422, 27.75, 76.56999969482422, 22.559999465942383, 37.5, 65.5, 56.41999816894531, 2.7100000381469727, 144.89999389648438, 160.0, 102.18000030517578, 36.72999954223633, 29.329999923706055, 162.38999938964844, 34.400001525878906, 62.060001373291016, 99.0, 99.0, 80.18000030517578, 122.0999984741211, 92.76000213623047, 52.52000045776367, 1.1299999952316284, 126.79000091552734, 44.40999984741211, 164.3699951171875, 12.760000228881836, 109.94000244140625, 126.29000091552734, 40.810001373291016, 81.3499984741211, 115.33999633789062, 44.25, 120.19000244140625, 159.72000122070312, 128.85000610351562, 92.30000305175781, 168.1100006103